In [13]:
import warnings
warnings.simplefilter('ignore')
import json
import pandas as pd
import numpy as np
from io import BytesIO
from zipfile import ZipFile
import gc
from tqdm import tqdm

In [8]:
train_text_path = './data/annotations/labeled.json'
test_text_path = './data/annotations/test_a.json'
unlabeled_text_path = './data/annotations/unlabeled.json'
save_path = './temp_data/'

In [20]:
# 拼接title_ocr_asr
def load_text_data(file_path, file_type, save_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = json.load(f)
    df_text = pd.DataFrame(text)
    all_ocr_text = []
    all_time = []
    for ocr in tqdm(df_text['ocr']):
        all_ocr_text.append(', '.join([i['text'] for i in ocr]))
        all_time.append([i['time'] for i in ocr])
    df_text['ocr_text'] = all_ocr_text
    df_text['ocr_time'] = all_time
    df_text['text'] =  df_text['title'].astype(str) + "[SEP]" + \
                       df_text['asr'].str[:60].astype(str) + df_text['asr'].str[60:].astype(str) + \
                       df_text['ocr_text'].str[:60].astype(str) + df_text['ocr_text'].str[60:].astype(str)
    if file_type == 'train':
        df_text = df_text[['text', 'category_id']]
        df_text.columns = ['text', 'label']
    elif file_type == 'unlabeled' or file_type == 'test':
        df_text = df_text[['text']]
    df_text.to_pickle(save_path + file_type + '_text.pkl')
    print(len(df_text))
    gc.collect()

In [21]:
load_text_data(train_text_path, file_type='train', save_path=save_path)
load_text_data(test_text_path, file_type='test', save_path=save_path)
load_text_data(unlabeled_text_path, file_type='unlabeled', save_path=save_path)

100%|██████████| 100000/100000 [00:00<00:00, 581836.75it/s]


100000


100%|██████████| 25000/25000 [00:00<00:00, 799969.48it/s]


25000


100%|██████████| 1000000/1000000 [00:03<00:00, 333216.60it/s]


1000000


In [17]:
df = pd.read_pickle('./temp_data/unlabeled_text.pkl')

0         166
1          81
2         273
3         727
4         125
         ... 
999995    534
999996     30
999997    235
999998     59
999999    351
Name: text, Length: 1000000, dtype: int64